In [11]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Subset
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
import torch.nn.functional as F


In [12]:
wandb.login(key='f56388c51b488c425a228537fd2d35e5498a3a91')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [13]:
import os
print(os.listdir("../input"))

[]


In [18]:
common_config = {
    "img_size": 224,
    "num_classes": 10,
    "batch_size": 32,
    "epochs": 10,
    "data_augmentation": True,
}

# Sweep config: we sweep over how many blocks to unfreeze, dropout, LR, and augmentation
sweep_config = {
    "method": "bayes",
    "metric": { "name": "val_acc", "goal": "maximize" },
    "parameters": {
        "freeze_before": { "values": [3, 5, 7, 9] },
        "dropout":        { "values": [0.2, 0.3, 0.4] },
        "learning_rate":  { "values": [1e-2, 1e-3, 1e-4] },
    }
}

sweep_id_partB = wandb.sweep(sweep_config, project="da6401_assignment2")
print("Sweep ID PartB:", sweep_id_partB)


Create sweep with ID: ilynlfke
Sweep URL: https://wandb.ai/da24m017-indian-institute-of-technology-madras/da6401_assignment2/sweeps/ilynlfke
Sweep ID PartB: ilynlfke


In [19]:
def prepare_data(config):
    # Data transforms
    train_tf = transforms.Compose([
        transforms.Resize((config["img_size"], config["img_size"])),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    val_tf = transforms.Compose([
        transforms.Resize((config["img_size"], config["img_size"])),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    
    # Paths (adjust as needed)
    train_path = "/kaggle/input/nature-12k/inaturalist_12K/train"
    test_path  = "/kaggle/input/nature-12k/inaturalist_12K/val"
    
    # Load and split
    full_train = datasets.ImageFolder(train_path, transform=train_tf)
    idx_by_class = {}
    for idx, (_,lbl) in enumerate(full_train.samples):
        idx_by_class.setdefault(lbl, []).append(idx)
    train_idx, val_idx = [], []
    for lbl, idxs in idx_by_class.items():
        np.random.shuffle(idxs)
        split = int(0.8*len(idxs))
        train_idx += idxs[:split]
        val_idx   += idxs[split:]
    train_ds = Subset(full_train, train_idx)
    
    # reuse same dataset object for val but with val_tf
    full_train.transform = val_tf
    val_ds = Subset(full_train, val_idx)
    
    train_loader = DataLoader(train_ds, batch_size=config["batch_size"],
                              shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_ds, batch_size=config["batch_size"],
                              shuffle=False, num_workers=4)
    test_ds      = datasets.ImageFolder(test_path, transform=val_tf)
    test_loader  = DataLoader(test_ds, batch_size=config["batch_size"],
                              shuffle=False, num_workers=4)
    
    return train_loader, val_loader, test_loader


In [20]:
class FineTuneEfficientNetV2(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters(config)
        
        # Load pretrained EfficientNet‑V2 small
        self.net = models.efficientnet_v2_s(pretrained=True)
        total_blocks = len(self.net.features)
        freeze_upto = total_blocks - config["freeze_before"]
        
        # Freeze all but last `freeze_before` blocks
        for idx, block in enumerate(self.net.features):
            for p in block.parameters():
                p.requires_grad = (idx >= freeze_upto)
        
        # Replace classifier with dropout + linear
        in_f = self.net.classifier[1].in_features
        self.net.classifier = nn.Sequential(
            nn.Dropout(config["dropout"]),
            nn.Linear(in_f, config["num_classes"])
        )
    
    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        x,y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc  = (logits.argmax(dim=1)==y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x,y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc  = (logits.argmax(dim=1)==y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
    
    def test_step(self, batch, batch_idx):
        x,y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc  = (logits.argmax(dim=1)==y).float().mean()
        self.log("test_loss", loss)
        self.log("test_acc", acc)
    
    def configure_optimizers(self):
        # Only unfrozen params get optimized
        params = filter(lambda p: p.requires_grad, self.parameters())
        return torch.optim.Adam(params, lr=self.hparams.learning_rate)


In [ ]:
# def train():
#     run = wandb.init()
#     sweep_cfg = run.config

#     # Merge defaults + sweep params
#     params = common_config.copy()
#     for k, v in sweep_cfg.items():
#         params[k] = v

#     # Name the run
#     run.name = f"e2v2_fb{params['freeze_before']}_do{params['dropout']}_lr{params['learning_rate']}_aug{params['data_augmentation']}"
#     run.save()

#     # Data loaders
#     train_loader, val_loader, test_loader = prepare_data(params)

#     # Logger & checkpoint
#     wandb_logger = WandbLogger(project="da6401_assignment2", log_model="all")
#     ckpt_cb = ModelCheckpoint(
#         monitor="val_acc",
#         mode="max",
#         save_top_k=1,
#         dirpath="finetune_ckpts",
#         filename=f"e2v2_fb{params['freeze_before']}_do{params['dropout']}_lr{params['learning_rate']}"  
#     )

#     # Model + Trainer
#     model   = FineTuneEfficientNetV2(params)
#     trainer = pl.Trainer(
#         max_epochs=params["epochs"],
#         logger=wandb_logger,
#         accelerator="gpu" if torch.cuda.is_available() else "cpu",
#         callbacks=[ckpt_cb]
#     )

#     trainer.fit(model, train_loader, val_loader)
#     trainer.test(model, test_loader)

#     # Save best checkpoint as artifact
#     best_path = ckpt_cb.best_model_path
#     print("Best model saved to:", best_path)

#     artifact = wandb.Artifact("efficientnetv2_best", type="model")
#     artifact.add_file(best_path)
#     run.log_artifact(artifact)

#     run.finish()


In [ ]:
# common_config = {
#     "img_size": 224,       # faster
#     "num_classes": 10,
#     "batch_size": 48,      # push it if you can
#     "epochs": 10,           # shorter sweep
#     "data_augmentation": True,
#     "learning_rate": 1e-4,
#     "dropout": 0.3,
    
    
# }

# sweep_config = {
#     "method": "grid",
#     "metric": { "name": "val_acc", "goal": "maximize" },
#     "parameters": {
#         "freeze_before":   { "values": [5, 7, 9] },
#     }
# }
# sweep_id_partB = wandb.sweep(sweep_config, project="da6401_assignment2")


In [21]:
def train():
    run    = wandb.init()
    sweep  = run.config
    params = {**common_config, **sweep}

    # Name the run
    run.name = f"{params['freeze_before']}_freeze_{params['dropout']}_dropout_{params['learning_rate']}_lr"
    run.save()

    # Data
    train_loader, val_loader, test_loader = prepare_data(params)

    # Wandb logger + callbacks
    wandb_logger = WandbLogger(project="da6401_assignment2", log_model="all")
    ckpt_cb = ModelCheckpoint(
        monitor="val_acc", mode="max", save_top_k=1,
        dirpath="finetune_ckpts",
        filename=f"ENETV2_{params['freeze_before']}_freeze_{params['dropout']}_dropout_{params['learning_rate']}_lr"
    )

    # Model
    model = FineTuneEfficientNetV2(params)

    # inside your train() function, after defining wandb_logger and ckpt_cb:

    trainer = pl.Trainer(
        max_epochs=params["epochs"],                              # e.g. 5 for your sweep
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        logger=wandb_logger,                                      # your WandbLogger instance
        callbacks=[ckpt_cb]                                       # your ModelCheckpoint callback
    )


    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, test_loader)

    # Log the best checkpoint as an artifact
    best_path = ckpt_cb.best_model_path
    artifact = wandb.Artifact("efficientnetv2_best", type="model")
    artifact.add_file(best_path)
    run.log_artifact(artifact)

    run.finish()


In [ ]:
# After defining sweep_id_partB and your train() function:
wandb.agent(sweep_id_partB, function=train, count=20)


wandb: Agent Starting Run: vqvlnfan with config:
wandb: 	dropout: 0.3
wandb: 	freeze_before: 7
wandb: 	learning_rate: 0.01


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 204MB/s]
wandb: WARNING Config item 'dro

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]